<a href="https://colab.research.google.com/github/balandongiv/mmocr_tutorial/blob/main/2_batch_process_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make Installation

In [1]:
%cd ..

!pip show torch

# Install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html

# Install mmdetection
!pip install mmdet

# # Install mmocr
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements.txt
!pip install -v -e .

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check mmocr installation
import mmocr
print(mmocr.__version__)

%cd /mmocr/
!ls

/
Name: torch
Version: 1.11.0+cu113
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.7/dist-packages
Requires: typing-extensions
Required-by: torchvision, torchtext, torchaudio, fastai
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html
     |████████████████████████████████| 37.7 MB 13.6 MB/s 
     |████████████████████████████████| 190 kB 6.7 MB/s 
     |████████████████████████████████| 1.4 MB 7.2 MB/s 
Cloning into 'mmocr'...
remote: Enumerating objects: 6551, done.
remote: Counting objects: 100% (484/484), done.
remote: Compressing objects: 100% (353/353), done.
remote: Total 6551 (delta 234), reused 268 (delta 124), pack-reused 6067
Receiving objects: 100% (6551/6551), 13.18 MiB | 25.65 MiB/s, done.
Resolving deltas: 100% (4012/4012), done.
/mmocr
     |███████████████████████

In [3]:
from mmocr.apis import init_detector
from mmocr.utils.model import revert_sync_batchnorm
import os

In [4]:
dconfig='/mmocr/configs'

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
det_config = '/mmocr/configs/textdet/textsnake/textsnake_r50_fpn_unet_1200e_ctw1500.py'

det_ckpt = 'https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_r50_fpn_unet_1200e_ctw1500-27f65b64.pth'

detect_model = init_detector(det_config, det_ckpt, device=device)
detect_model = revert_sync_batchnorm(detect_model)

load checkpoint from http path: https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_r50_fpn_unet_1200e_ctw1500-27f65b64.pth


Downloading: "https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_r50_fpn_unet_1200e_ctw1500-27f65b64.pth" to /root/.cache/torch/hub/checkpoints/textsnake_r50_fpn_unet_1200e_ctw1500-27f65b64.pth


  0%|          | 0.00/139M [00:00<?, ?B/s]

In [8]:
recog_config='/mmocr/configs/textrecog/sar/sar_r31_parallel_decoder_academic.py'

recog_ckpt='https://download.openmmlab.com/mmocr/textrecog/sar/sar_r31_parallel_decoder_academic-dba3a4a3.pth'

recog_model = init_detector(recog_config, recog_ckpt, device=device)
recog_model = revert_sync_batchnorm(recog_model)

load checkpoint from http path: https://download.openmmlab.com/mmocr/textrecog/sar/sar_r31_parallel_decoder_academic-dba3a4a3.pth


Downloading: "https://download.openmmlab.com/mmocr/textrecog/sar/sar_r31_parallel_decoder_academic-dba3a4a3.pth" to /root/.cache/torch/hub/checkpoints/sar_r31_parallel_decoder_academic-dba3a4a3.pth


  0%|          | 0.00/219M [00:00<?, ?B/s]

/mmocr/mmocr/apis/inference.py:50: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [9]:
dpath='/mmocr/tests/data/toy_dataset/imgs/test/img_1.jpg'

In [11]:
from mmocr.apis.inference import model_inference
# Send args and models to the MMOCR model inference API
# and call post-processing functions for the output
end2end_res = []

# Find bounding boxes in the images (text detection)
img='tests/data/toy_dataset/imgs/test/img_1.jpg'


arr=mmcv.imread(dpath)
det_result = model_inference(detect_model, img,ann=None,
                         batch_mode=False,
                         return_data=False)

det_result=[det_result]

/usr/local/lib/python3.7/dist-packages/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
/mmocr/mmocr/models/textdet/postprocess/utils.py:51: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  canvas = canvas[1:h + 1, 1:w + 1].astype(np.bool)


In [12]:
arr=mmcv.imread(dpath)

In [15]:
from mmocr.datasets.pipelines.crop import crop_img
bboxes_list = [res['boundary_result'] for res in det_result]
bboxes=bboxes_list[0]
filename='test.img'
img_e2e_res = {}
img_e2e_res['filename'] = filename
img_e2e_res['result'] = []
box_imgs = []
for bbox in bboxes:
    box_res = {}
    box_res['box'] = [round(x) for x in bbox[:-1]]
    box_res['box_score'] = float(bbox[-1])
    box = bbox[:8]
    if len(bbox) > 9:
        min_x = min(bbox[0:-1:2])
        min_y = min(bbox[1:-1:2])
        max_x = max(bbox[0:-1:2])
        max_y = max(bbox[1:-1:2])
        box = [min_x, min_y, max_x, min_y, max_x, max_y, min_x, max_y]
    box_img = crop_img(arr, box)

    box_imgs.append(box_img)

sa

In [16]:
arrays=box_imgs
batch_size=3
recog_results = []
n = batch_size
arr_chunks = [arrays[i:i + n] for i in range(0, len(arrays), n)
              ]
for chunk in arr_chunks:
    recog_results.extend(model_inference(recog_model, chunk, batch_mode=True))



/mmocr/mmocr/apis/utils.py:53: UserWarning: Remove "MultiRotateAugOCR" to support batch inference since samples_per_gpu > 1.
  warnings.warn(warning_msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/mmocr/mmocr/apis/utils.py:53: UserWarning: Remove "MultiRotateAugOCR" to support batch inference since samples_per_gpu > 1.
  warnings.warn(warning_msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [18]:
print(recog_results[0])

{'text': 'Carpark', 'score': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}


In [ ]:
for i, recog_result in enumerate(recog_results):
    text = recog_result['text']
    text_score = recog_result['score']
    if isinstance(text_score, (list, tuple)):
        text_score = sum(text_score) / max(1, len(text))
    img_e2e_res['result'][i]['text'] = text
    img_e2e_res['result'][i]['text_score'] = text_score